<a href="https://colab.research.google.com/github/purang2/BERT/blob/main/%ED%98%B8%EB%B0%98%EC%9A%B0_1008_5CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets #from huggingface
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 14.5 MB/s 
     |████████████████████████████████| 163 kB 84.7 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 13.8 MB/s 
     |████████████████████████████████| 212 kB 57.1 MB/s 
     |████████████████████████████████| 115 kB 59.7 MB/s 
     |████████████████████████████████| 127 kB 84.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.2 MB/s 


In [ ]:
from datasets import load_dataset
import pandas as pd 

data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"


train_data = pd.read_csv(data_path+"/train_data.tsv", sep='\t')
train_labels = pd.read_csv(data_path+"/train_labels.csv")

train_data

,sentence_id,person,sentence,scene,context
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는"
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대
...,...,...,...,...,...
7334,12285,청난,종남이 맛있는것도 사주고 잘 챙겨줘.,S3576,NaN
7335,12286,건강,걱정마라. 먹어도 같이 먹고 굶어도 같이 굶는다.,S3576,NaN
7336,12287,종남,아빠. 우리 저거 타.,S3576,NaN
7337,12288,건강,"나 못타는데. 한번도 안타봤어. 툭하면 차멀미까지 하는데, 봐라, 너도 내몸땡이에 ...",S3576,NaN


In [ ]:
train_drop = train_data[['person', 'sentence', 'context']]
train_drop['label'] = train_labels['label']
train_pair = train_drop

import numpy as np 

train_pair['context2'] = train_pair['context'].fillna('none')
train_pair['sentence2'] = train_pair['sentence'].fillna('none')

train_pair


train_pair = train_pair[['person', 'context2', 'sentence2', 'label']]
train_pair['scene'] = train_data['scene']
train_pair

train = list()
scene_df = train_pair['scene']
scene_df = scene_df.drop_duplicates().reset_index(drop=True)


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(0<i and i<len(df)-1 ):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2']+"\""+ " " + df.iloc[i]['person'] + "가 말하길 " +"\""+df.iloc[i]['sentence2'] +"\""+" "+df.iloc[i+1]['person'] + "가 말하길 " +"\""+ df.iloc[i+1]['sentence2']+"\""
    return scenario
  elif(i == len(df)-1):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2'] +"\""+" "+ df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario


def trans_df(train_pair, train):
  for i in scene_df[:]:
    temp = train_pair[train_pair['scene']==i].reset_index(drop=True)
    train+=[mk_scenario(j, temp) for j in range(len(temp))]
    
trans_df(train_pair, train)
train = pd.DataFrame(train, columns=('scenario',))
print(train.iloc[1]['scenario'])

train['label'] = train_pair['label']
train
train_pairs = train

labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

train_pairs['label'] = [labels[label] for label in train_pairs['label']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep> 어영가 말하길 "우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다."


In [ ]:
train_pairs = pd.read_csv(data_path+'/AEDA_aug_train_data.csv', encoding='utf-8')
train_pairs

,scenario,label
0,"none <sep> 어영가 말하길 ""야! 전화 받아. 아무리 바빠도 내전화는 받아야...",2
1,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는 <sep>...",2
2,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,1
3,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,2
4,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구...,0
...,...,...
14673,"none , < : sep> 건강가 말하길 ""행복할 ; 일도 많다. 이까짓 ! 거 ...",0
14674,"none < : sep> ? 청난가 말하길 ""종남 ? 이 맛있는것도 사주고 ? 잘 ...",0
14675,"none <sep> 건강가 말 ? 하길 . ""걱정마라. 먹어도 : 같이 먹고 굶어도...",0
14676,", none <sep> 종남 : 가 ? 말하길 ""아빠. 우리 저거 타."" ! 건강가...",0


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from datasets import Dataset
import pandas as pd 
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import evaluate


max_input_length = 200
#tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    #metric = evaluate.load('glue', 'qnli')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    #return metric.compute(predictions=predictions, references=labels)
    return metric.compute(predictions=predictions, references=labels, average='micro')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-discriminator", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=3)
#klue/roberta-large

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'cla

In [ ]:
#Gather your training arguments~
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    do_train=True,
    do_eval=True,
#    per_device_train_batch_size=32,
#    per_device_eval_batch_size=32,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    #eval_steps=50,
    warmup_steps=500,
    weight_decay=0.01,
    #logging_steps=50, 
    #load_best_model_at_end=True,
    #gradient_accumulation_steps=64,  # total number of steps before back propagation
)

trainer = Trainer(
        model=model,
        args=training_args,
        #train_dataset = train_dataset,
        #eval_dataset= val_dataset,
        data_collator=data_collator, 
        tokenizer=tokenizer,  
        compute_metrics = compute_metrics  
    )

def preprocess_function(examples):
    inputs = [doc for doc in examples["scenario"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,padding='max_length')

    return model_inputs


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario
  elif(0<i and i<len(df)-1 ):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2']+"\""+ " " + df.iloc[i]['person'] + "가 말하길 " +"\""+df.iloc[i]['sentence2'] +"\""+" "+df.iloc[i+1]['person'] + "가 말하길 " +"\""+ df.iloc[i+1]['sentence2']+"\""
    return scenario
  elif(i == len(df)-1):
    scenario = scenario + df.iloc[i-1]['person'] + "가 말하길 " +"\""+ df.iloc[i-1]['sentence2'] +"\""+" "+ df.iloc[i]['person'] + "가 말하길 " +"\""+ df.iloc[i]['sentence2'] +"\""
    return scenario


def trans_df2(test_pair, test):
  for i in scene_df2[:]:
    temp = test_pair[test_pair['scene']==i].reset_index(drop=True)
    test+=[mk_scenario(j, temp) for j in range(len(temp))]



k_fold_logits = []

kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for train_df_idx, val_df_idx in kfold.split(train_pairs, train_pairs['label']):

    model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=3)

    trainer = Trainer(
            model=model,
            args=training_args,
            #train_dataset = train_dataset,
            #eval_dataset= val_dataset,
            data_collator=data_collator, 
            tokenizer=tokenizer,  
            compute_metrics = compute_metrics  
        )
    

    train_df = train_pairs.iloc[train_df_idx]
    val_df = train_pairs.iloc[val_df_idx]


    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)

    train_dataset =train_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])
    val_dataset = val_dataset.map(preprocess_function, batched=True).remove_columns(["__index_level_0__",'scenario'])

    #Collect your model, training args, dataset, data collator, tokenizer in Trainer
    trainer.train_dataset = train_dataset
    trainer.eval_dataset = val_dataset
    trainer.train()


    data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"

    test_data = pd.read_csv(data_path+"/public_test_data.tsv", sep='\t')

    test_pair = test_data[['sentence_id','context','sentence','person', 'scene']]

    test_pair['context2'] = test_pair['context'].fillna('none')
    test_pair['sentence2'] = test_pair['sentence'].fillna('none')
    #모든 글자 합치기
    test = list()
    scene_df2 = test_pair['scene']
    scene_df2 = scene_df2.drop_duplicates().reset_index(drop=True)
        
    trans_df2(test_pair, test)
    test = pd.DataFrame(test, columns=('scenario',))
    print("--------------------\n", test)

    #test_pairs = test_pair[['label','new']]
    test['sentence_id'] = test_pair['sentence_id']

    test_dataset = Dataset.from_pandas(test)
    test_dataset =test_dataset.map(preprocess_function, batched=True).remove_columns(['scenario'])


    predictions = trainer.predict(test_dataset)

    k_fold_logits.append(predictions.predictions)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--roberta-large/snapshots/5193b95701189160c45d02a1033a4ea55bdbe259/config.json
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11742
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1468
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.728600,0.500081,0.802793
2,0.447200,0.316644,0.900886


...

Epoch,Training Loss,Validation Loss,F1
1,0.727100,0.427822,0.836116
2,0.449700,0.313392,0.900852


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2935
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2935
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value i

--------------------
                                                scenario
0     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
1     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
2     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
3     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
4     음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...
...                                                 ...
2561  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 순...
2562  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 우...
2563  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 건...
2564  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 종...
2565  가방든 현찰우미. 애들 신났고. 순경건강 나와선. 부러운듯 보는 종남 <sep> 건...

[2566 rows x 1 columns]


  0%|          | 0/3 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence_id. If sentence_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2566
  Batch size = 16


In [ ]:
'''
data_path ="drive/MyDrive/호반우NLP_한국인감정/KERC22Dataset/KERC22Dataset_PublicTest"

test_data = pd.read_csv(data_path+"/public_test_data.tsv", sep='\t')

test_pair = test_data[['sentence_id','context','sentence','person', 'scene']]

test_pair['context2'] = test_pair['context'].fillna('none')
test_pair['sentence2'] = test_pair['sentence'].fillna('none')
#모든 글자 합치기
test = list()
scene_df2 = test_pair['scene']
scene_df2 = scene_df2.drop_duplicates().reset_index(drop=True)


def mk_scenario(i, df):
  scenario = df.iloc[0]['context2']+" <sep> " 
  if(i==0):
    scenario = scenario + df.iloc[i]['sentence2']
    return scenario
  elif(0<i and i<len(df)-1 ):
    scenario = scenario + df.iloc[i-1]['sentence2'] + df.iloc[i]['sentence2'] + df.iloc[i+1]['sentence2']
    return scenario
  elif(i == len(df)-1):
    scenario = scenario + df.iloc[i-1]['sentence2'] + df.iloc[i]['sentence2'] 
    return scenario


def trans_df2(test_pair, test):
  for i in scene_df2[:]:
    temp = test_pair[test_pair['scene']==i].reset_index(drop=True)
    test+=[mk_scenario(j, temp) for j in range(len(temp))]
    
trans_df2(test_pair, test)
test = pd.DataFrame(test, columns=('scenario',))
print("--------------------\n", test)

#test_pairs = test_pair[['label','new']]
test['sentence_id'] = test_pair['sentence_id']

test_dataset = Dataset.from_pandas(test)
test_dataset =test_dataset.map(preprocess_function, batched=True).remove_columns(['scenario'])

'''

predict= k_fold_logits[0] + k_fold_logits[1] + k_fold_logits[2] +k_fold_logits[3] +k_fold_logits[4]


list1 = []
list2 = []


for i in range(2566):
  list1.append(i)
  list2.append(-1)



raw_data = {'Id': list1,
            'Predicted': list2
            }
df = pd.DataFrame(raw_data)



labels = {'euphoria':0,
          'neutral':1,
          'dysphoria':2
          }

df['Predicted'] = np.argmax(predict, axis=-1)

d = ['euphoria','neutral','dysphoria']

df['Predicted'] = [d[label] for label in df['Predicted']]


df['Id'] = test_dataset['sentence_id']

#x = [d[labels[label]] for label in ['euphoria','euphoria','dysphoria','neutral']]

df



df.to_csv('drive/MyDrive/호반우NLP_한국인감정/submission_CV_1008_5CV_Seed24.csv', index=False)

df

,Id,Predicted
0,10,euphoria
1,11,neutral
2,12,neutral
3,13,euphoria
4,14,euphoria
...,...,...
2561,12256,euphoria
2562,12257,euphoria
2563,12258,euphoria
2564,12259,euphoria


In [ ]:
df['Predicted'].value_counts()

dysphoria    1649
euphoria      641
neutral       276
Name: Predicted, dtype: int64